In [ ]:
%matplotlib notebook
%pylab

## Timing and accuracy results for DG, CG and WPA
<hr style="border-width:4px; border-color:coral"></hr>

In [ ]:
import pandas 

In [ ]:
idx = pandas.IndexSlice

# Walltime     : time TFinal (global time)
# Prep         : one time step up time
# Integration  : 
# time steps   : Number of time steps to time tfinal
# dt           : Time step size
# CFL          : CFL number
# Errors       : (1-norm, 2-norm and inf-norm) errors

ex_list = ['advection','SWE']

cols = ['walltime', 'Prep', 'Integ', 'time steps', 'dt', 'cfl','1-norm','2-norm','inf-norm']

order = [1,2,3,4,6]
elements = [8,16,32,64,128,256,512,1024]
iterables = [ex_list, ['DG','CG','WPA'],['exact','inexact','w/lim','wout/lim'], order, elements]

index = pandas.MultiIndex.from_product(iterables,names=['example','method','integration','order','elements'])
df = pandas.DataFrame(index=index,columns=cols).sort_index()

df.loc[idx['Advec','CG',['exact','inexact']]]

## Read in binary WPA data
<hr style="border-width:2px; border-color:black"></hr>

Read in data created during simulation.  The data is output in binary using a Numpy dynamic type `dtype`.  The data has to then be read in using the same format.  The file structure is : 

    Row 1  : nlen               Number of N values in convergence study

The following block then repeats four times for order=(1,2), limiter=(0,1)

Block :

      dtype([('order',int),('limiter',int)])   
      
      # Followed by N rows of 
      dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])
                  


In [ ]:
output_file = 'WPA/wpa_adv_data.dat'

# Open file as a binary file
fout = open(output_file,"rb")  
nlen = fromfile(fout,dtype=int,count=1)

# Create a data type for storing results;  use 'd' for double precision entries
dt_data = dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])  

# Create a nested data type
dt_info = dtype([('order',int),('limiter',int)])
dt_block = dtype([('info',dt_info),('data',(dt_data,nlen))])

# Read in four sets of data (orders 1,2 and limiters 0,1)
wpa_data = fromfile(fout,dtype=dt_block, count=4)
fout.close()

cols = ['walltime','Prep','Integ','time steps','dt', 'cfl', '1-norm','2-norm','inf-norm']

# Store data in DataFrame
for k in range(4):     # order=(1,2) x lim=(0,1)
    order = wpa_data[k]['info']['order']
    lim = wpa_data[k]['info']['limiter']
    if lim == 0:        
        limidx = 'wout/lim'
    else:
        limidx = 'w/lim'
    
    for d in wpa_data[k]['data']:
        N = d[0]
        a = [d[i] for i in range(1,len(d))]
        df.loc[idx['advection','WPA',limidx,order,N],cols] = a


# Format the data when displaying the DataFrame

fstr = {'walltime' : "{:.4e}".format, 
        'Prep' : "{:.2e}".format,
        "dt" : "{:.4e}", 
        'Integ' : "{:.4e}".format,
        'cfl' : "{:.2f}".format, 
        '1-norm' : '{:.4e}'.format,
        '2-norm' : '{:.4e}'.format,
        'inf-norm' : '{:.4e}'.format}
        
df_wpa = df.loc[idx['advection','WPA',['w/lim','wout/lim'],1:2],:]
df_wpa.style.format(fstr)

## Read in binary CG/DG data
<hr style="border-width:2px; border-color:black"></hr>

Read in data created during simulation.  The data is output in binary using a Numpy dynamic type `dtype`.  The data has to then be read in using the same format.  The file structure is : 

    Row 1  : nlen               Number of N values in convergence study

The following block then repeats four times for order=(1,2), limiter=(0,1)

Block :

      dtype([('order',int),('limiter',int)])   
      
      # Followed by N rows of 
      dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])
                  


In [ ]:
output_file = 'CGDG/dg_adv_data_RK4_implicit2.dat'

# Open file as a binary file
fout = open(output_file,"rb")  
nlen = fromfile(fout,dtype=int,count=1)

# Create a data type for storing results;  use 'd' for double precision entries
dt_data = dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])  

# Create a nested data type
dt_info = dtype([('order',int),('integType',int)])
dt_block = dtype([('info',dt_info),('data',(dt_data,nlen))])

# Read in four sets of data (orders 1,2 and limiters 0,1)
dg_data = fromfile(fout,dtype=dt_block, count=10)
fout.close()

cols = ['walltime','Prep','Integ','time steps','dt', 'cfl', '1-norm','2-norm','inf-norm']

# Store data in DataFrame
for k in range(10):     # order=(1,2) x lim=(0,1)
    order = dg_data[k]['info']['order']
    Itype = dg_data[k]['info']['integType']
    if Itype == 1:        
        typex = 'inexact'
    else:
        typex = 'exact'
    
    for d in dg_data[k]['data']:
        N = d[0]
        a = [d[i] for i in range(1,len(d))]
        df.loc[idx['advection','DG',typex,order,N],cols] = a


# Format the data when displaying the DataFrame

fstr = {'walltime' : "{:.4e}".format, 
        'Prep' : "{:.2e}".format,
        "dt" : "{:.4e}", 
        'Integ' : "{:.4e}".format,
        'cfl' : "{:.2f}".format, 
        '1-norm' : '{:.4e}'.format,
        '2-norm' : '{:.4e}'.format,
        'inf-norm' : '{:.4e}'.format}
        
df_dg = df.loc[idx['advection','DG',['exact','inexact'],:],:]
df_dg.style.format(fstr)

In [ ]:
output_file = 'CGDG/cg_adv_data_RK4_implicit2.dat'

# Open file as a binary file
fout = open(output_file,"rb")  
nlen = fromfile(fout,dtype=int,count=1)

# Create a data type for storing results;  use 'd' for double precision entries
dt_data = dtype([('N',int),('wall','d'), ('setup','d'),('integ','d'),('M',int),('dt','d'),('cfl','d'),
                  ('1-norm','d'),('2-norm','d'),('inf-norm','d')])  

# Create a nested data type
dt_info = dtype([('order',int),('integType',int)])
dt_block = dtype([('info',dt_info),('data',(dt_data,nlen))])

# Read in four sets of data (orders 1,2 and integType 1,2)
cg_data = fromfile(fout,dtype=dt_block, count=10)

fout.close()

cols = ['walltime','Prep','Integ','time steps','dt', 'cfl', '1-norm','2-norm','inf-norm']

# Store data in DataFrame
for k in range(10):                  # order = (1,2) x integType = (1,2)
    order = cg_data[k]['info']['order']
    Itype = cg_data[k]['info']['integType']
    if Itype == 1:        
        typex = 'inexact'
    else:
        typex = 'exact'
    
    for d in cg_data[k]['data']:
        N = d[0]
        a = [d[i] for i in range(1,len(d))]
        df.loc[idx['advection','CG',typex,order,N],cols] = a


# Format the data when displaying the DataFrame

fstr = {'walltime' : "{:.4e}".format, 
        'Prep' : "{:.2e}".format,
        "dt" : "{:.4e}", 
        'Integ' : "{:.4e}".format,
        'cfl' : "{:.2f}".format, 
        '1-norm' : '{:.4e}'.format,
        '2-norm' : '{:.4e}'.format,
        'inf-norm' : '{:.4e}'.format}
        
df_cg = df.loc[idx['advection','CG',['exact','inexact'],:],:]
df_cg.style.format(fstr)

## Add extra columns
<hr style="border-width:2px; border-color:black"></hr>

In [ ]:
# Add a DOF column

integration = df.index.levels[2].values
orders = df.index.levels[3].values
elements = df.index.levels[4].values

for o in orders:
    # Is this right?  
    for I in integration:
        df.loc[idx['advection','DG',I,o,:],'DOF'] = (o + 1)*array(elements)
        df.loc[idx['advection','CG',I,o,:],'DOF'] = o*array(elements) + 1
    
# One degree of freedom per element
df.loc[idx['advection','WPA','w/lim',1,:],'DOF'] = elements
df.loc[idx['advection','WPA','w/lim',2,:],'DOF'] = elements

# display data frame
df.loc[idx['advection','DG','exact',:,:],:]

## Using a MultiIndex table

<hr style="border-width:4px; border-color:coral"/>

In [ ]:
# Extract all of the data for one example
df.loc['advection']

In [ ]:
df.loc[('advection','WPA',('w/lim','wout/lim')),:]

In [ ]:
df.loc[('advection','DG',('exact')),['walltime','Prep','1-norm','cfl']]


In [ ]:
cols = ['walltime','Prep','time steps','1-norm'] 
df.loc[idx['advection',:,('exact','wout/lim')],cols].unstack(3)

In [ ]:
#df.loc[idx['advection',('DG','CG'),'exact',2],:].\
#       style.background_gradient(subset=['walltime'],cmap='YlOrBr')

## Plotting : CG vs. DG
<hr style="border-width:4px; border-color:coral"></hr>

In [ ]:
# set up nice tick marks for log data
def set_xticks(P):
    p0 = log2(P[0])
    p1 = log2(P[-1])
    xlim([2**(p0-1), 2**(p1+1)])
    
    Pstr = (['{:d}'.format(int(p)) for p in P])
    xticks(P,Pstr)

In [ ]:
figure(1)
#subplot(121)
#clf()
orders = array([1,2,3,4,6])
order_wpa = 2
# Retrieve element values - level 3 in the Multiindex
Nv = df.index.levels[4].values

symb1 = ['r.-','r*-','r.--','-rx','-r^']
symb2 = ['b.-','b*-','b.--','-bx','-b^']

for i,o in enumerate(orders):
    
    ntimecg = df.loc[idx['advection','CG','exact',o,:],'time steps'].values
    ntimedg = df.loc[idx['advection','DG','exact',o,:],'time steps'].values
    time_dge = df.loc[idx['advection','DG','exact',o,:],'walltime'].values
    time_cge = df.loc[idx['advection','CG','exact',o,:],'walltime'].values
    
    loglog(Nv,time_cge/ntimecg,symb1[i],markersize=6,label='CG:N = {}'.format(o))
    loglog(Nv,time_dge/ntimedg,symb2[i],markersize=6,label='DG:N = {}'.format(o))
    
time_wpaw = df.loc[idx['advection','WPA','w/lim',order_wpa,:],'walltime'].values
ntimewpaw = df.loc[idx['advection','WPA','w/lim',order_wpa,:],'time steps'].values
loglog(Nv,time_wpaw/ntimewpaw,'g.-',markersize=6,label='WPA')
ylabel('walltime')
xlabel('# Elements')
title('Exact, w/lim')

#ylim([1e-4,1e5])
set_xticks(Nv)
legend()
show()
figure(10)
#subplot(122)
for i,o in enumerate(orders):
    
    ntimecg = df.loc[idx['advection','CG','inexact',o,:],'time steps'].values
    ntimedg = df.loc[idx['advection','DG','inexact',o,:],'time steps'].values
    
    time_dgi = df.loc[idx['advection','DG','inexact',o,:],'walltime'].values
    time_cgi = df.loc[idx['advection','CG','inexact',o,:],'walltime'].values
    
    loglog(Nv,time_cgi/ntimecg,symb1[i],markersize=6,label='CG:N = {}'.format(o))
    loglog(Nv,time_dgi/ntimedg,symb2[i],markersize=6,label='DG:N = {}'.format(o))
    
time_wpano = df.loc[idx['advection','WPA','wout/lim',order_wpa,:],'walltime'].values
ntimewpano = df.loc[idx['advection','WPA','wout/lim',order_wpa,:],'time steps'].values
loglog(Nv,time_wpano/ntimewpano,'g.-',markersize=6,label='WPA')
ylabel('walltime')
xlabel('# Elements')
title('Inexact, wout/lim')

#ylim([1e-4,1e5])
set_xticks(Nv)
legend()
show()
ntimewpano

In [ ]:
figure(2)
#subplot(121)
#clf()
orders = array([1,2,3,4,6])
order_wpa = 2
# Retrieve element values - level 3 in the Multiindex
Nv = df.index.levels[4].values

symb1 = ['r.-','r*-','r.--','-rx','-r^']
symb2 = ['b.-','b*-','b.--','-bx','-b^']


for i,o in enumerate(orders):
    
    
    time_dge = df.loc[idx['advection','DG','exact',o,:],'walltime'].values
    time_cge = df.loc[idx['advection','CG','exact',o,:],'walltime'].values
    
    e2_dge = df.loc[idx['advection','DG','exact',o,:],'2-norm'].values
    e2_cge = df.loc[idx['advection','CG','exact',o,:],'2-norm'].values
    
    loglog(time_cge, e2_dge,symb1[i],markersize = 6,label='CG:N = {}'.format(o))
    loglog(time_dge, e2_cge,symb2[i],markersize = 6,label='DG:N = {}'.format(o))
    
time_wpaw = df.loc[idx['advection','WPA','w/lim',order_wpa,:],'walltime'].values
e2_wpaw = df.loc[idx['advection','WPA','w/lim',order_wpa,:],'2-norm'].values
loglog(time_wpaw,e2_wpaw,'g.-',markersize=6,label='WPA(W/lim)')
ylabel('L2_error')
xlabel('walltime')
title('Exact, w/lim')
ylim([1e-14,1e0])
xlim([1e-6, 1e3])
#set_xticks(Nv)
legend()
show()

In [ ]:
figure(11)
for i,o in enumerate(orders):
    
    time_dgi = df.loc[idx['advection','DG','inexact',o,:],'walltime'].values
    time_cgi = df.loc[idx['advection','CG','inexact',o,:],'walltime'].values
    
    e2_dgi = df.loc[idx['advection','DG','inexact',o,:],'2-norm'].values
    e2_cgi = df.loc[idx['advection','CG','inexact',o,:],'2-norm'].values
    
    loglog(time_cgi, e2_dgi,symb1[i],markersize = 6,label='CG:N = {}'.format(o))
    loglog(time_dgi, e2_cgi,symb2[i],markersize = 6,label='DG:N = {}'.format(o))

    
time_wpano = df.loc[idx['advection','WPA','wout/lim',order_wpa,:],'walltime'].values
e2_wpano = df.loc[idx['advection','WPA','wout/lim',order_wpa,:],'2-norm'].values
loglog(time_wpano,e2_wpano,'g.-',markersize=6,label='WPA(wout/lim)')
ylabel('L2_error')
xlabel('walltime')
title('Inexact, wout/lim')
ylim([1e-14,1e0])
xlim([1e-6, 1e3])
#set_xticks(Nv)
legend()
show()

In [ ]:
figure(3)
clf()   

Nv = df.index.levels[4].values
symb1 = ['r.-','r*-','r.--','-rx','-r^']
symb2 = ['b.-','b*-','b.--','-bx','-b^']

for i,o in enumerate(orders):
    
    e2_dge = df.loc[idx['advection','DG','exact',o,:],'2-norm'].values
    e2_cge = df.loc[idx['advection','CG','exact',o,:],'2-norm'].values
    
    # DG exact 
    if(o >= 4):
        p = polyfit(log(Nv[:3]), log(e2_dge.astype(float)[:3]), 1)
    else:
        p = polyfit(log(Nv[:]), log(e2_dge.astype(float)[:]), 1)

    loglog(Nv, e2_dge, symb1[i],markersize=10, label = 'DG:N = {}, rate = {:.2f}'.format(o,p[0]))
    #loglog(Nv, exp(polyval(p,log(Nv))), 'k:')

    # CG exact
    if(o >= 4):
        p = polyfit(log(Nv[:3]), log(e2_cge.astype(float)[:3]), 1)
    else:
        p = polyfit(log(Nv[:]), log(e2_cge.astype(float)[:]), 1)
        
    loglog(Nv, e2_cge, symb2[i],markersize=10, label = 'CG:N = {}, rate = {:.2f}'.format(o,p[0]))
    #loglog(Nv, exp(polyval(p,log(Nv))), 'g:')
    

# WPA w/lim
e2_wpaw = df.loc[idx['advection','WPA','w/lim',order_wpa,:],'2-norm'].values
p = polyfit(log(Nv), log(e2_wpaw.astype(float)), 1)
loglog(Nv, e2_wpaw, '.-',markersize=10, label = 'WPA, rate = {:.2f}'.format(p[0]))
loglog(Nv, exp(polyval(p,log(Nv))), '--')

set_xticks(Nv)
xlabel('# Elements')
ylabel('Error (L2-error)')
title('Error vs number of Elements (exact, w/lim)')

grid(axis='both',linestyle='--')
legend()
show()   

In [ ]:
figure(4)
clf()   

Nv = df.index.levels[4].values
symb1 = ['r.-','r*-','r.--','-rx','-r^']
symb2 = ['b.-','b*-','b.--','-bx','-b^']

for i,o in enumerate(orders):
    
    e2_dgi = df.loc[idx['advection','DG','inexact',o,:],'2-norm'].values
    e2_cgi = df.loc[idx['advection','CG','inexact',o,:],'2-norm'].values
    
    # DG exact 
    if(o >= 3):
        p = polyfit(log(Nv[:3]), log(e2_dgi.astype(float)[:3]), 1)
        
    else:
        p = polyfit(log(Nv[:]), log(e2_dgi.astype(float)[:]), 1)

    loglog(Nv, e2_dgi, symb1[i],markersize = 10, label = 'DG:N = {}, rate = {:.2f}'.format(o,p[0]))
    #loglog(Nv, exp(polyval(p,log(Nv))), 'k:')

    # CG exact
    if(o >= 3):
        p = polyfit(log(Nv[:3]), log(e2_cgi.astype(float)[:3]), 1)
    else:
        p = polyfit(log(Nv[:]), log(e2_cgi.astype(float)[:]), 1)
        
    loglog(Nv, e2_cgi, symb2[i],markersize = 10, label = 'CG:N = {}, rate = {:.2f}'.format(o,p[0]))
    #loglog(Nv, exp(polyval(p,log(Nv))), 'g:')
    

# WPA w/lim
e2_wpano = df.loc[idx['advection','WPA','wout/lim',order_wpa,:],'2-norm'].values
p = polyfit(log(Nv), log(e2_wpano.astype(float)), 1)
loglog(Nv, e2_wpano, '.-',markersize = 10, label = 'WPA, rate = {:.2f}'.format(p[0]))
loglog(Nv, exp(polyval(p,log(Nv))), '--')

set_xticks(Nv)
xlabel('# Elements')
ylabel('Error (L2-error)')
title('Error vs number of Elements (inexact, wout/lim)')

grid(axis='both',linestyle = '--')
legend()
show()   